In [38]:
%load_ext autoreload
%autoreload 2

import warnings; warnings.simplefilter("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from node import Node, OldHome, PowerPlant
from grid import EnergyGrid

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Overview
I want to model the dynamics of a microgrid. Particularly interested in allowing asynchronous updating, meaning that we need an OOP model, rather than a matrix-based one (which would require full-grid timestepping). The trouble with this is that it makes this process much harder to parallelize

Resources:
- NE Real time energy usage dashboard: https://www.eia.gov/beta/electricity/gridmonitor/dashboard/electric_overview/regional/REG-NE

## Try running it
I've put together an ```EnergyGrid``` class to facilitate running these nodes. Since it operates over timesteps, we don't take advantage of our ability to update asynchronously, but it makes things much cleaner here.

In [39]:
# Grab some jittery points in the Boston area
lat, lon, mu = 42.477642, -71.130318, 0.1
get_pos = lambda lat, lon, mu: [np.random.normal(lon, mu), np.random.normal(lat, mu)]

generate_randomly = False
if generate_randomly:
    capacities = np.append(1000, np.zeros(10))
    nodes = np.array([
        Node(get_pos(lat, lon, mu), storage_capacity=c) for c in capacities
    ])
    
else:
    nodes = np.array([
        OldHome(get_pos(lat, lon, mu)), 
        OldHome(get_pos(lat, lon, mu)), 
        OldHome(get_pos(lat, lon, mu)), 
        OldHome(get_pos(lat, lon, mu)), 
        PowerPlant(get_pos(lat, lon, mu)),
        PowerPlant(get_pos(lat, lon, mu))
    ])
    
# We can optionally provide a resistance matrix, telling what the electrical
# cost is for directional transmission between nodes.
resistance_matrix = np.zeros((len(nodes), len(nodes)))
np.fill_diagonal(resistance_matrix, 1)

    
grid = EnergyGrid(nodes)
grid.make_neighbors(resistance_matrix=resistance_matrix)

# MVP Demo
grid.nodes[0].step(new_demand=10, new_supply=0)
grid.nodes[1].step(new_demand=10, new_supply=0)

In [40]:
grid.run_simulation()

478919 pulled 0.28186285228047075 watts from node 870722
449801 pulled 0.29524726491073255 watts from node 870722
478919 pulled 0.2839658015843227 watts from node 870722
449801 pulled 0.29598798221908573 watts from node 870722
478919 pulled 0.30750363037231015 watts from node 870722
449801 pulled 0.30659877488047405 watts from node 870722
478919 pulled 0.2726124487763905 watts from node 870722
449801 pulled 0.25207397677584475 watts from node 870722
478919 pulled 0.2843439408198184 watts from node 870722
449801 pulled 0.2951826179584286 watts from node 870722
478919 pulled 0.27356340157106296 watts from node 870722
449801 pulled 0.2974695573943429 watts from node 870722
478919 pulled 0.3171995536268996 watts from node 870722
449801 pulled 0.2981022254200399 watts from node 870722
478919 pulled 0.31584343563652406 watts from node 870722
449801 pulled 0.3012114837494678 watts from node 870722
478919 pulled 0.37575926430383083 watts from node 870722
449801 pulled 0.3093060188720968 watts 

In [43]:
grid.nodes[0].get_transaction_logs(t_end=pd.to_datetime("2019-07-10 08:00:00"))

,source,dest,amount
timestamp,,,
2019-07-01 00:00:00,<node.OldHome object at 0x8320ee590>,<node.OldHome object at 0x831a5a7d0>,0.281863
2019-07-01 00:00:00,<node.OldHome object at 0x8320ee590>,<node.OldHome object at 0x831a5a790>,0.295247
2019-07-01 01:00:00,<node.OldHome object at 0x8320ee590>,<node.OldHome object at 0x831a5a7d0>,0.283966
2019-07-01 01:00:00,<node.OldHome object at 0x8320ee590>,<node.OldHome object at 0x831a5a790>,0.295988
2019-07-01 02:00:00,<node.OldHome object at 0x8320ee590>,<node.OldHome object at 0x831a5a7d0>,0.307504
...,...,...,...
2019-07-10 04:00:00,<node.PowerPlant object at 0x831a5a850>,<node.OldHome object at 0x8320ee590>,0.293153
2019-07-10 05:00:00,<node.PowerPlant object at 0x831a5a850>,<node.OldHome object at 0x8320ee590>,0.319258
2019-07-10 06:00:00,<node.PowerPlant object at 0x831a5a850>,<node.OldHome object at 0x8320ee590>,0.335228


In [42]:
grid.visualize()

KeyError: "None of ['timestamp'] are in the columns"